# The `Bayes Theorem` 

In [1]:
import pandas as pd
import numpy as np

Hatırlıyor musun? Derste bu teoremi işlemiştik.

*Bayes Teoremi, koşullu olasılık hesaplamanı sağlar.
*Makine Öğrenmesinde sıkça, bilgini güncellemek için kullanılır.
*Basit bir formülü olmasına rağmen çoğu zaman beklenmedik içgörüler ortaya çıkarır.

🧑🏻‍🏫  `Bayes Theorem` nedir?  [Brilliant](https://brilliant.org/wiki/bayes-theorem/) :

>Bayes Teoremi, verilen bir kanıta (Data) dayanarak hipotezlerin (A) olasılıklarını nasıl güncelleyeceğimizi açıklayan bir formüldür.


🧮 Formül şu şekildedir:

$$ \mathbb{P}(A | Data) =  \mathbb{P}(A) \times \frac{\mathbb{P}(Data | A) }{\mathbb{P}(Data)}$$

## 0) Challenge Bağlamı: Beklenen hava durumuna göre dışarıda spor yapmalı mıyız?

* Bu challenge’da, bu formülü yeniden hesaplamaya çalışacağız.
* `Hava koşulları ` (Rain, Sunny, Overcast) ve `play` (Yes, No) yani hava durumuna göre bir spor maçının oynanması gerekip gerekmediğini gösteren bir veri setimiz var.

In [2]:
weather_data_example = ['Sunny', 'Overcast', 'Rainy', 'Sunny', 'Sunny', 'Overcast', 'Rainy', 'Rainy', 'Sunny',
'Rainy', 'Sunny', 'Overcast', 'Overcast', 'Rainy']

play_data_example = ['No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No']

data = {'weather': weather_data_example, 'play': play_data_example}

df = pd.DataFrame(data = data)
df

,weather,play
0,Sunny,No
1,Overcast,Yes
2,Rainy,Yes
3,Sunny,Yes
4,Sunny,Yes
5,Overcast,Yes
6,Rainy,No
7,Rainy,No
8,Sunny,Yes
9,Rainy,Yes


🚀 Hadi Hesaplayalım $ \large P(play \mid weather) = P(play) \times \frac{P(weather \mid play)}{P(weather)} $

## 1)Isınma: bir `frequency table` ile sayıları anlamak

✍️ Bir kalem , kağıt al ve  **`frequency table`** ı tamamla:

| Weather  | Played | No | Total |
|----------|------|----|-------|
| Sunny    |     |   |      |
| Overcast |     |   |      |
| Rainy    |     |   |      |
| Total    |     |   |   14  |

<details>
    <summary>Cevap Buraya</summary>
    
| Weather  | Played | No | Total |
|----------|------|----|-------|
| Sunny    | 3    | 2  | 5     |
| Overcast | 4    | 0  | 4     |
| Rainy    | 2    | 3  | 5     |
| Total    | 9    | 5  | 14    |     
</details>

## 2) Prior probability : $ \mathbb{P}(play)$

🤔 Bir oyunun oynanmasının teorik olasılığı nedir ❓

Bunu, önceki tablondaki sayılara bakarak bulabilirsin.

<details>
    <summary>Cevap</summary>
    
| Weather  | Played | No | Total |
|----------|------|----|-------|
| Total    | 9    | 5  | 14    |     
    
$ \mathbb{P}(played) = \frac{9}{14} = 64.29 \% $
</details>

👩🏻‍💻 `prior_probability` fonksiyonunu yazarak sonucu hesapla.

In [14]:
def prior_probability(event_name: str, observations: list) -> float:
    
    return (df["play"]==event_name).sum()/len(df)
    
# 👀 KODUNU test etmek için aşağıdakini çalıştır.
#Eğer hiçbir şey görünmüyorsa, fonksiyonun doğru çalışıyor demektir.Aksi halde, kodunu düzeltmek için inceleme yap!
assert(round(prior_probability("Yes", play_data_example),4) == 0.6429)

☝️ Bilgin olsun (FYI): Bu garip görünen gösterimler
```python
def prior_probability(event_name: bool, observations: list) -> float:
```
**typing hints** (tip ipuçları) olarak adlandırılır.

Python’da zorunlu değildir, ancak fonksiyonun hangi türde argümanlar alması ve hangi türde değer döndürmesi gerektiğini okuyucuya belirtmek için kullanılır.

Ayrıca bu tip ipuçlarını zorunlu kılan ve tipler uymazsa hata veren Python kütüphaneleri de vardır.
Büyük projelerde sessizce hata oluşmasını engellemek için bu tip ipuçlarını kullanmak iyi bir pratiktir.

## 3) Likelihood :  $ \mathbb{P}(weather | play)$

🤔 Bir oyunun OYNANMAMIŞ olduğunu bildiğimizde havanın yağmurlu olma olasılığı nedir?

Bunu önceki frekans tablosundan okuyabilirsin.

<details>
    <summary>Cevap</summary>
    
| Weather  | No | 
|----------|----|
| Sunny    | 2  | 
| Overcast | 0  | 
| Rainy    | 3  | 
| Total    | 5  |         
    
$ \mathbb{P}(play) = \frac{3}{5} = 60 \% $
</details>

In [51]:
def likelihood(weather, played, weather_data, play_data):
    d=0
    p=0
    for  w,p1 in zip(weather_data, play_data):
      if p1==played:
          p+=1
          if w==weather:
             d+=1
    if p == 0:
        return 0

    return d / p
# 👀 Aşağıdakini çalıştırarak kodunu test et.
#Eğer hiçbir şey görünmüyorsa fonksiyonun doğru çalışıyor demektir.Bir çıktı görünürse, kodunu gözden geçirip düzeltmelisin!
assert(likelihood("Rainy", "No", weather_data_example, play_data_example) == 0.60)

## 4) Posterior probability : $ \large P(play \mid weather ) = P(play) \times \frac{P(weather \mid play)}{P(weather)} $

🔥 Sonunda posterior olasılığı hesaplayabiliriz:

$$\large \text{posterior probability} = \text{prior probability} \times \text{likelihood} \times \beta $$ 

where $ \large \beta = \frac{1}{P(weather)} $ is a _normalization factor_.
 
<details>
    <summary>Beklenen sonuçlar</summary>

Önceden doldurduğun tabloyu hatırlıyor musun?

| Weather  | Played | No | Total |
|----------|------|----|-------|
| Sunny    | 3    | 2  | 5     |
| Overcast | 4    | 0  | 4     |
| Rainy    | 2    | 3  | 5     |
| Total    | 9    | 5  | 14    |   
    
Bu tabloya göre $ \mathbb{P}(played | weather) $ şöyle:
    
| Weather  | Proba(Played\|Weather) | Proba(No\|Weather) |
|----------|----------------------|--------------------|
| Sunny    | 3/5 = 0.6                  | 2/5 = 0.4                |
| Overcast | 4/4 = 1.0                  | 0/4 = 0.0                |
| Rainy    | 2/5 = 0.4                  | 3/5 = 0.6                |
    
</details>

In [55]:
def posterior_probability(played, weather, weather_data, play_data):
    m=0
    n=0
    b=0
    for wd, pd in zip(weather_data, play_data):
        if wd==weather:
            m+=1
            if pd==played:
                n+=1
    if m==0:
        return 0

    return n/m
            

In [56]:
# 👀 Aşağıdakini çalıştırarak kodunu test et.
assert(np.isclose(posterior_probability("Yes", "Sunny", weather_data_example, play_data_example), 0.6))
assert(np.isclose(posterior_probability("No", "Sunny", weather_data_example, play_data_example), 0.4))
assert(np.isclose(posterior_probability("Yes", "Overcast", weather_data_example, play_data_example), 1.0))
assert(np.isclose(posterior_probability("No", "Overcast", weather_data_example, play_data_example), 0))
assert(np.isclose(posterior_probability("Yes", "Rainy", weather_data_example, play_data_example), 0.4))
assert(np.isclose(posterior_probability("No", "Rainy", weather_data_example, play_data_example), 0.6))

## 5) Geriye dönüp neler yaptığını anlamak

Bu meydan okuma boyunca öğrendiklerin sayesinde, şu soruları cevaplayabilir misin :

1. _"Hava güneşliyse maçların oynanma olasılığı oynanmama olasılığından daha yüksektir" 👉 Bu ifade doğru mu?
2. Bir sonraki maç sırasında yağmur yağacağını kesin olarak biliyorsan 🤔, maçın iptal edilme olasılığı için en iyi tahminin (olasılık olarak) nedir?

In [61]:
a=posterior_probability("Yes", "Sunny", weather_data_example, play_data_example)
b=posterior_probability("No", "Sunny", weather_data_example, play_data_example)
print(a)
print(b)

0.6
0.4


🏁 Tebrikler, artık `Bayes formula`nın nasıl çalıştığı hakkında çok daha iyi bir fikrin var! 

💾 Not defterini `git add/commit/push` unutma.

📆 Bu konsepte tekrar döneceğiz, özellikle şu modüllerde:
* `Decision Science - Inferential Stastistics`
* `Machine Learning - Performance Metrics - Confusion Matrix`
* `Natural Language Processinsg`


▶️ Meraklıysan veya sabırsızsan, derste de bahsettiğimiz 3Blue1Brown’un 15 dakikalık YouTube videosunu şimdiden izleyebilirsin: [15-min Youtube video from 3Blue1Brown](https://www.youtube.com/watch?v=HZGCoVF3YvM) 